In [ ]:
# ЧЕРНОВОЙ ВАРИАНТ ФОРМИРОВАНИЯ ДИАЛОГОВ для ИНКОРПОРИРОВАНИЯ из ТАБЛИЦ МАСЕЛ

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import datetime
import numpy as np
import openpyxl
import re
import requests
import threading, time
import json
import os

from bs4 import BeautifulSoup



In [ ]:
def savetxt(dir, name, text):
  if not os.path.exists('result'): os.mkdir('result')
  if not os.path.exists('result/'+dir): os.mkdir('result/'+dir)
  with open('result/'+dir+'/'+name,'w',encoding='utf-8') as f:
    f.write(text)
  return True

def resulttotext(r):
  if r is not None:
    return r.text+'.'
  else:
    return ''

def parser(soup):
    text = ''
    pages = soup.findAll('div', class_='g-padding')
    if pages is None:
      print(soup.find('title').text.split(' – ')[0])
      print(soup)
    else:
      for page in pages:
        if page.find('div',class_='faq') is not None:
          text += page.find('div',class_='faq').text
        else:
          content = page.select('div[class*="text-"]')
          text += ''.join(i.text+'\n' for i in content)
    return text



In [ ]:
# Pars page whith models

pagemodels = requests.get('https://www.kia.ru/models/').text
modellinks = []

html = ''.join(line.strip() for line in pagemodels.split("\n"))
soup = BeautifulSoup(html, "html.parser")

cards_list = soup.find_all('div', class_='car-card')

for card in cards_list:
  modellinks.append('https://www.kia.ru'+ str(card.a['href']))


In [ ]:
def getmodelsoup(url):
  model = requests.get(url).text
  html = ''.join(line.strip() for line in model.split("\n"))
  return BeautifulSoup(html, "html.parser")

modeldict = {}

for i in modellinks:
  modeldict[i] = getmodelsoup(i)


In [ ]:
def stringstolist(div):
  '''Превращаем блок в список строк'''
  lis = []
  try:
    for string in div.strings:
      if string not in [' ','']:
        lis.append(string.text)
  except:
    lis = ['']
  return lis


def parsermodel(soup):
  text = ''

  # Обрабатываю по id basic
  basic = soup.select('div[id*="basic_"]')
  for div in basic:
    # Перевожу строки в список, для удобного обращиния
    textlist = stringstolist(div)
    try:
      img = div.find('img')['data-src']
    except:
      img = ''
    text += f'## {textlist[0]}\n{". ".join(textlist[1:])}\nФото {img}\n\n'

  # Обрабатываю по id dizain
  dizain = soup.select('div[id*="dizain_"]')
  for div in dizain:
    if len(div['id'].split('_')) > 2: continue # Пропускаем внутренние блоки
    # Перевожу строки в список, для удобного обращиния
    textlist = stringstolist(div)[:-1]
    text += f'## {textlist[0]}\n{". ".join(textlist[1:3])}\n'
    for t in textlist[3:]:
      text += t+'. '
    text = text[:-2]+'\n\n'

  # Обрабатываю по id eksterer
  eksterer = soup.select('div[id*="eksterer_"]')
  for div in eksterer:
    if len(div['id'].split('_')) > 2: continue # Пропускаем внутренние блоки
    # Перевожу строки в список, для удобного обращиния
    textlist = stringstolist(div)[:-1]
    text += f'## {textlist[0]}\n{". ".join(textlist[1:3])}\n'
    for t in textlist[3:]:
      text += t+'. '
    text = text[:-2]+'\n\n'

  # Обрабатываю по id interer
  interer = soup.select('div[id*="interer_"]')
  for div in interer:
    if len(div['id'].split('_')) > 2: continue # Пропускаем внутренние блоки
    # Перевожу строки в список, для удобного обращиния
    textlist = stringstolist(div)[:-1]
    text += f'## {textlist[0]}\n{". ".join(textlist[1:3])}\n'
    for t in textlist[3:]:
      text += t+'. '
    text = text[:-2]+'\n\n'

  # Обрабатываю по id style
  style = soup.select('div[id*="style_"]')
  for div in style:
    if len(div['id'].split('_')) > 2: continue # Пропускаем внутренние блоки
    # Перевожу строки в список, для удобного обращиния
    textlist = stringstolist(div)[:-1]
    text += f'## {textlist[0]}\n{". ".join(textlist[1:3])}\n'
    for t in textlist[3:]:
      text += t+'. '
    text = text[:-2]+'\n\n'

  # Обрабатываю по id multimedia
  multimedia = soup.select('div[id*="multimedia_"]')
  for div in multimedia:
    if len(div['id'].split('_')) > 2: continue # Пропускаем внутренние блоки
    # Перевожу строки в список, для удобного обращиния
    textlist = stringstolist(div)[:-1]
    text += f'## {textlist[0]}\n{". ".join(textlist[1:3])}\n'
    for t in textlist[3:]:
      text += t+'. '
    text = text[:-2]+'\n\n'

  # Обрабатываю по id tehnologii
  tehnologii = soup.select('div[id*="tehnologii_"]')
  for div in tehnologii:
    if len(div['id'].split('_')) > 2: continue # Пропускаем внутренние блоки
    # Перевожу строки в список, для удобного обращиния
    textlist = stringstolist(div)[:-1]
    text += f'## {textlist[0]}\n{". ".join(textlist[1:3])}\n'
    for t in textlist[3:]:
      text += t+'. '
    text = text[:-2]+'\n\n'

  # Обрабатываю по id bezopasnost
  bezopasnost = soup.select('div[id*="bezopasnost_"]')
  for div in bezopasnost:
    if len(div['id'].split('_')) > 2: continue # Пропускаем внутренние блоки
    # Перевожу строки в список, для удобного обращиния
    textlist = stringstolist(div)[:-1]
    text += f'## {textlist[0]}\n{". ".join(textlist[1:3])}\n'
    for t in textlist[3:]:
      text += t+'. '
    text = text[:-2]+'\n\n'

  # Обрабатываю по id teplye_opcii
  teplye_opcii = soup.select('div[id*="teplye_opcii_"]')
  for div in teplye_opcii:
    if len(div['id'].split('_')) > 3: continue # Пропускаем внутренние блоки
    # Перевожу строки в список, для удобного обращиния
    textlist = stringstolist(div)[:-1]
    text += f'## {textlist[0]}\n{". ".join(textlist[1:3])}\n'
    for t in textlist[3:]:
      text += t+'. '
    text = text[:-2]+'\n\n'

  # Обрабатываю по id komfort
  komfort = soup.select('div[id*="komfort_"]')
  for div in komfort:
    if len(div['id'].split('_')) > 2: continue # Пропускаем внутренние блоки
    # Перевожу строки в список, для удобного обращиния
    textlist = stringstolist(div)[:-1]
    text += f'## {textlist[0]}\n{". ".join(textlist[1:3])}\n'
    for t in textlist[3:]:
      text += t+'. '
    text = text[:-2]+'\n\n'

  # Обрабатываю по id vmestimost
  vmestimost = soup.select('div[id*="vmestimost_"]')
  for div in vmestimost:
    if len(div['id'].split('_')) > 2: continue # Пропускаем внутренние блоки
    # Перевожу строки в список, для удобного обращиния
    textlist = stringstolist(div)[:-1]
    text += f'## {textlist[0]}\n{". ".join(textlist[1:3])}\n'
    for t in textlist[3:]:
      text += t+'. '
    text = text[:-2]+'\n\n'

  # Обрабатываю по id dvigatel
  dvigatel = soup.select('div[id*="dvigatel_"]')
  for div in dvigatel:
    if len(div['id'].split('_')) > 2: continue # Пропускаем внутренние блоки
    # Перевожу строки в список, для удобного обращиния
    textlist = stringstolist(div)[:-1]
    text += f'## {textlist[0]}\n{". ".join(textlist[1:3])}\n'
    text += 'Модели двигателей: '
    ul = div.find('ul')
    for t in stringstolist(ul):
      text += t+', '
    text = text[:-2]+'\n\n'

  return text

def parseroptions(url):
  text = '## Комплектации\n'
  links = []
  url = url.replace('desc','options')
  suop = getmodelsoup(url)
  ahrefs = suop.findAll('div',class_='config__variants__slide')
  for a in ahrefs:
    cont = a.find('li').text
    url = a.find('a')
    option = getmodelsoup('https://www.kia.ru'+url['href'])
    titel = option.find('title').text
    text += f'### Комплектация: {titel}\nЦена: {cont}\n'
    info = option.findAll('div', class_="info-section")

    for i in info:
      t2 = i.find('div', class_="info-section__header").text
      text += t2 + ': '
      if t2.strip() == 'Технические характеристики' or t2.strip() == 'Спецификация':
        dl = i.findAll('dl')
        for j in dl:
          text += j.find('dt').text + ': ' + j.find('dd').text + '; '
      else:
        li = i.findAll('li')
        for j in li:
          text += j.text + ', '
      text = text[:-2] + '\n'

  return text

def resultsmodel(link):

  print(link)
  name = modeldict[link].find('title').text.split(' – ')[0]
  name = name.replace('/','-')
  text = f'#  {name} - [link {link}]\n'
  text += parsermodel(modeldict[link])
  text += parseroptions(link)
  savetxt('models', name+'.txt', text)
  print('Done '+ name)


threads = []
# Добавляю потоки с функцией сохранения в файл в список потоков
for link in modellinks:
  threads.append(threading.Thread(target=resultsmodel, args=(link,)))



In [ ]:
# Технологии

def parserabout(soup):
    text = '## '
    pages = soup.find('div', class_='articles-detail__technology-txt')
    if pages is None:
      print(soup.find('title').text.split(' – ')[0])
      print(soup)
    else:
        content = pages.select('div[class*="text-"]')
        text += ''.join(i.text+'\n' for i in content)
    return text

url = "https://www.kia.ru/ajax/page/technologies/more?limit=45&page=1"

headers = {
    "Referer": "https://www.kia.ru/about/technologies/",  # Example Referer header
}

response = requests.get(url, headers=headers)
json_data = response.text

data = json.loads(json_data)

ids = [tech['id'] for tech in data['content']['technologies']]

static_url = 'https://www.kia.ru/about/technologies/'
urls = []

for id in ids:
    urls.append(static_url+id)

def resultsabout(link):

  print(link)
  content = requests.get(link)
  if content.status_code == 200:
    #constructor-block
    text = ''
    html = ''.join(line.strip() for line in content.text.split("\n"))
    soup = BeautifulSoup(html, "html.parser")
    name = soup.find('title').text.split(' – ')[0]
    text += parserabout(soup)
  savetxt('about', name.replace('/','-')+'.txt', text)
  print('Done '+ name)
print(urls)
for link in urls:

# Добавляю потоки с функцией сохранения в файл в список потоков
  threads.append(threading.Thread(target=resultsabout, args=(link,)))

In [ ]:
# Сбор с закладки "Журнал"

url = "https://www.kia.ru/ajax/page/mediacenter/magazine/more?limit=100&page=1"
static_url = "https://www.kia.ru/press/magazine/"
HEADERS = {"Referer": static_url}

response = requests.get(url=url, headers=HEADERS)
json_data = response.text
data = json.loads(json_data)
all_article_list = []

for article in data["content"]["media_center"]["magazine"]:
    code = article["code"]
    all_article_list.append(static_url + code + '/')

url = "https://www.kia.ru/ajax/page/mediacenter/news/more?limit=100&page=1"
static_url = "https://www.kia.ru/press/news/"
response = requests.get(url=url, headers=HEADERS)
json_data = response.text
data = json.loads(json_data)
all_article_list = []

for article in data["content"]["media_center"]["news"]:
    code = article["code"]
    all_article_list.append(static_url + code + '/')

def resultspress(link):

  print(link)
  content = requests.get(link)
  if content.status_code == 200:
    #constructor-block
    text = ''
    html = ''.join(line.strip() for line in content.text.split("\n"))
    soup = BeautifulSoup(html, "html.parser")
    name = soup.find('title').text.split(' – ')[0]
    head_press = soup.h1.text.strip() #заголовок статьи
    all_img_list, all_img = [], ""
    try:
      [all_img_list.append(img.find("img").attrs.get("src")) for img in soup.find_all("div", class_="articles-detail__content__offset")]  # все фото из статьи
      for img in all_img_list:
        all_img += img + ","
    except:
      all_img = ""

    try:
      date_press = soup.select_one("div.articles-detail__date").text.strip()  # дата статьи
      for val in soup.find_all("div", class_="g-container"):
        for child in val.children:
          if date_press in child.text:
            text += f"## {head_press}>\n"
            text += child.text.replace("\xa0", " ")
            text += f"\nФотографии из статьи: {all_img[:-1]}\n\n"
    except:
      for val in soup.find_all("div", class_="g-container"):
        for child in val.children:
          if child.find("h1") is not None and child.find("h1") != -1:
            text += f"## {head_press}\n"
            text += child.text.replace("\xa0", " ")
            text += f"\nФотографии из статьи: {all_img[:-1]}\n\n"
    if text != '':
      savetxt('press', name.replace('/','-')+'.txt', text)
      print('Done '+ name)
#threads = []
for link in all_article_list:
  threads.append(threading.Thread(target=resultspress, args=(link,)))





In [ ]:
 # Start all threads
for x in threads:
  x.start()
  time.sleep(0.5)

 # Wait for all of them to finish
for x in threads:
  x.join()

In [ ]:
url = "https://www.kia.ru/kiaflex/"
response = requests.get(url=url, headers=HEADERS).text
html = ''.join(line.strip() for line in response.split("\n"))
soup = BeautifulSoup(html, "html.parser")

savetxt('kiaflex', 'kiaflex.txt', '# '+parser(soup))

In [ ]:
#Тест ассортимента

base_url = "https://www.kia.ru/ajax/page/accessories/filter?sort=sort&order=desc&page=1"


headers = {
    "Referer": "https://www.kia.ru/service/accessories/",  # Example Referer header
}

bigdata = []


start_page = 1

while True:
    current_url = base_url + str(start_page)
    response = requests.get(current_url, headers=headers)
    json_data = response.text
    data = json.loads(json_data)
    if len(data['content']['accessories']) == 0: break



    for tech in data['content']['accessories']:
      bigdata.append(tech)

    start_page += 1

text = '# Аксесуары\n'
for i in bigdata:
  if i['material'] == '':
    i['material'] = {}
    i['material']['name'] = ''
  text += f'## Наименование: {i["name"].strip()}, Фото: https://cdn.kia.ru/resize/1295x632{i["image"]},'
  text += f'Артикул: {i["article"]}, Материал: {i["material"]["name"].strip()}.\n{i["text"]}'
  if i['technical_features'] is not None:
    html = ''.join(line.strip() for line in i['technical_features'].split("\n"))
    soup = BeautifulSoup(html, "html.parser")
    technical_features = '. '.join(soup.strings)
  text += technical_features + '\n'
savetxt('accessories', 'accessories.txt', text)

In [ ]:

# Test oil page
oil_res = {}
headers={"Referer": 'https://www.kia.ru/'}

json_data = json.loads(requests.get('https://www.kia.ru/ajax/decoder/model_lines',
                         headers={"Referer": 'https://www.kia.ru/'}).text)
model_lines = list(json_data['content']['model_lines'])
print(datetime.datetime.now())
for id in model_lines:
  try:
    model_line_id = id['id']
    json_data = json.loads(requests.get('https://www.kia.ru/ajax/decoder/years',
                              params={'model_line_id': model_line_id},
                              headers=headers).text)
    years = list(json_data['content']['years'])
    for year in years:
        year = year['id']
        json_data = json.loads(requests.get('https://www.kia.ru/ajax/decoder/models',
                                params={'model_line_id': model_line_id, 'year': year},
                                headers=headers).text)
        models = list(json_data['content']['models'])
        for model in models:
          model_id = model['id']
          json_data = json.loads(requests.get('https://www.kia.ru/ajax/decoder/complectations',
                                params={'model_id': model_id},
                                headers=headers).text)
          complectations = list(json_data['content']['complectations'])
          for complectation in complectations:
            complectation_id = complectation['id']
            json_data = json.loads(
                          requests.get('https://www.kia.ru/ajax/page/oils/complectations/'+complectation_id,
                          headers=headers).text)
            oils = list(json_data['content']['oils'])
            car = json_data['content']['car']
            if oils != []:
              for oil in oils:
                com_name = car['model']['model_line']['name'] + ' '
                com_name += car['model']['generation']['name'] +'/ '+str(year)+'/ '
                com_name += car['model']['carcass']['name'] + '/ '
                com_name += str(car['modification']['engine']['engine_volume']) + ' '
                com_name += car['modification']['engine']['engine_type'] + '/ '
                com_name += car['modification']['engine']['fuel_type'] + '/ '
                com_name += car['modification']['transmission']['drive'] + '/ '
                com_name += car['modification']['transmission']['gearbox'] + '/ '

                if not oil['name'] in oil_res.keys():
                  oil_res[oil['name']] = [oil['description'],com_name]
                else:
                  oil_res[oil['name']] += [com_name]
  except Exception as e:
    print(e)
print(oil_res)
print(datetime.datetime.now())

In [ ]:
for oil in oil_res:
  print(oil,oil_res[oil][0])

In [ ]:
!echo "# Технологии" > database.txt
!cat result/about/*.txt >> database.txt
!cat result/press/*.txt >> database.txt
!cat result/models/*.txt >> database.txt
!cat result/kiaflex/*.txt >> database.txt
!cat result/accessories/*.txt >> database.txt
!zip -r result.zip result database.txt
!zip database.zip database.txt

In [ ]:
!rm -rf /content/drive/MyDrive/KIA_PARSER

In [ ]:
!mkdir -p /content/drive/MyDrive/KIA_PARSER

In [ ]:

!cp -f database.zip /content/drive/MyDrive/KIA_PARSER/database.zip

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
OILS_TEXT="""
# Масло для двигателя
Масло для двигателя
## Наименование: REPSOL ELITE EVOLUTION LONG LIFE C3/SN 5W30
REPSOL ELITE EVOLUTION LONG LIFE C3/SN 5W30
Предназначено для использования в высокофорсированных бензиновых и дизельных двигателях, оборудованных сажевым фильтром. Благодаря высокому качеству, данное масло выделяется низким образованием отложений. Снижает трение, защищает двигатель от износа и имеет большую стойкость к окислению, что позволяет сохранять все свойства в течение длительного срока.
Подходит моделям с комплектациями: Carnival KA4/ 2022/ кроссвэн (7 мест)/ 2.2 CRDi/ Дизель/ Передний/ Автомат/ , Carnival KA4/ 2022/ кроссвэн (8 мест)/ 2.2 CRDi/ Дизель/ Передний/ Автомат/ , Carnival KA4/ 2022/ кроссвэн (7 мест)/ 2.2 CRDi/ Дизель/ Передний/ Автомат/ , Carnival KA4/ 2022/ кроссвэн (8 мест)/ 2.2 CRDi/ Дизель/ Передний/ Автомат/ , Carnival KA4/ 2021/ кроссвэн (7 мест)/ 2.2 CRDi/ Дизель/ Передний/ Автомат/ , Carnival KA4/ 2021/ кроссвэн (8 мест)/ 2.2 CRDi/ Дизель/ Передний/ Автомат/ , Carnival KA4/ 2021/ кроссвэн (7 мест)/ 2.2 CRDi/ Дизель/ Передний/ Автомат/ , Carnival KA4/ 2021/ кроссвэн (8 мест)/ 2.2 CRDi/ Дизель/ Передний/ Автомат/ , Ceed CD PE/ 2022/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD PE/ 2021/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD/ 2021/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD/ 2021/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD/ 2019/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Ceed CD/ 2019/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Ceed CD/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD FL/ 2018/ хэтчбэк 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd JD FL/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2018/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD FL/ 2018/ хэтчбэк 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd JD FL/ 2016/ хэтчбэк 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd JD FL/ 2016/ хэтчбэк 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2016/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2016/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2016/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD FL/ 2015/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2015/ хэтчбэк 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2015/ хэтчбэк 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd JD FL/ 2015/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2015/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD/ 2015/ хэтчбэк 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd JD/ 2015/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD/ 2015/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2015/ хэтчбэк 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2015/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2014/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD/ 2014/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2014/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2014/ хэтчбэк 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2014/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2014/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2014/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD/ 2014/ хэтчбэк 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2013/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2013/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2013/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD/ 2013/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD/ 2013/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2013/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2012/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2012/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD/ 2012/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2012/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2012/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD/ 2012/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2019/ универсал 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd SW JD FL/ 2018/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD FL/ 2018/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2018/ универсал 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd SW JD FL/ 2018/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd SW JD FL/ 2016/ универсал 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd SW JD FL/ 2016/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2016/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD FL/ 2016/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2015/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2015/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD FL/ 2015/ универсал 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd SW JD FL/ 2015/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2015/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD/ 2015/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2015/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2015/ универсал 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd SW JD/ 2014/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2014/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD/ 2014/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2014/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2014/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD/ 2014/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2013/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2013/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2013/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD/ 2013/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2013/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD/ 2012/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2012/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD/ 2012/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed SW CD/ 2022/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed SW CD/ 2021/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Ceed SW CD/ 2021/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed SW CD/ 2021/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato BD/ 2022/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato BD/ 2022/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato BD/ 2022/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato BD/ 2022/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato BD/ 2021/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2021/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2021/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2016/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2016/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2015/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2015/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2015/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2014/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2014/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2014/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2014/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2014/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2014/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2014/ купе/ 2.0 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2014/ купе/ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2013/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2013/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2013/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2013/ купе/ 2.0 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2013/ купе/ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2013/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2013/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2013/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2022/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2021/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2021/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , K900 RJ/ 2022/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2021/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2021/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2020/ седан 4 дв./ 5.0 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2020/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2020/ седан 4 дв./ 5.0 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2020/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2019/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2019/ седан 4 дв./ 5.0 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2019/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2018/ седан 4 дв./ 5.0 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2018/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , Mohave HM PE2/ 2022/ кроссовер 5 дв. (6 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE2/ 2022/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE2/ 2022/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE2/ 2021/ кроссовер 5 дв. (6 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE2/ 2021/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE2/ 2021/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE2/ 2021/ кроссовер 5 дв. (6 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE2/ 2021/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE2/ 2020/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM FL/ 2020/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE/ 2019/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM FL/ 2019/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM FL/ 2018/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM FL/ 2018/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM FL/ 2017/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM FL/ 2017/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Optima JF FL/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2020/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2020/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF PE/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF PE/ 2020/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF PE/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF PE/ 2020/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2019/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2019/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2019/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF FL/ 2019/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2019/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF FL/ 2019/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2019/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2019/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2018/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF FL/ 2018/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2018/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2018/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2018/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF/ 2018/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2018/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2018/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF/ 2017/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF/ 2017/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2016/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF/ 2016/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2016/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2016/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA PE/ 2020/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Механика/ , Picanto JA PE/ 2020/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA PE/ 2019/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA/ 2019/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA/ 2019/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA/ 2018/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA/ 2018/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA/ 2017/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA/ 2017/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD FL/ 2018/ хэтчбэк 3 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , ProCee'd JD FL/ 2018/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD FL/ 2018/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2018/ хэтчбэк 3 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , ProCee'd JD FL/ 2017/ хэтчбэк 3 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2017/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2017/ хэтчбэк 3 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , ProCee'd JD FL/ 2017/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD FL/ 2017/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2017/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD FL/ 2017/ хэтчбэк 3 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , ProCee'd JD FL/ 2017/ хэтчбэк 3 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2016/ хэтчбэк 3 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2016/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2016/ хэтчбэк 3 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , ProCee'd JD FL/ 2016/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD FL/ 2016/ хэтчбэк 3 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2015/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2015/ хэтчбэк 3 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , ProCee'd JD FL/ 2015/ хэтчбэк 3 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2015/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD/ 2015/ хэтчбэк 3 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD/ 2015/ хэтчбэк 3 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , ProCee'd JD/ 2015/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD/ 2015/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD/ 2014/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD/ 2014/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD/ 2014/ хэтчбэк 3 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , ProCee'd JD/ 2014/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD/ 2014/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD/ 2014/ хэтчбэк 3 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , ProCee'd JD/ 2013/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD/ 2013/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD/ 2013/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB PE/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB PE/ 2021/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB PE/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB PE/ 2021/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB PE/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB PE/ 2020/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB PE/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB PE/ 2020/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2020/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2020/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2019/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2019/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2019/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2019/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2018/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2018/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2018/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2018/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2017/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2017/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2017/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2017/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2017/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2017/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2016/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2016/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2016/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2016/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2016/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2016/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2016/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2016/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2016/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2016/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2016/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2016/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2015/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2015/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2015/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2015/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2015/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2015/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2015/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2015/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2015/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2015/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2015/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2015/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2014/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2014/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2014/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2014/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X FB PE/ 2021/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X FB PE/ 2021/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X FB PE/ 2021/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X FB PE/ 2021/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X FB PE/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X FB PE/ 2020/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X FB PE/ 2020/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X FB PE/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2020/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2020/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2017/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2017/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Sorento MQ4/ 2022/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Робот/ , Sorento MQ4/ 2022/ кроссовер 5 дв. (7 мест)/ 2.2 CRDi/ Дизель/ Полный/ Робот/ , Sorento MQ4/ 2021/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Робот/ , Sorento MQ4/ 2021/ кроссовер 5 дв. (7 мест)/ 2.2 CRDi/ Дизель/ Полный/ Робот/ , Sorento MQ4/ 2021/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Робот/ , Sorento MQ4/ 2021/ кроссовер 5 дв. (7 мест)/ 2.2 CRDi/ Дизель/ Полный/ Робот/ , Sorento XM FL/ 2021/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2021/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2021/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2021/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2021/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2021/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento MQ4/ 2020/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Робот/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento MQ4/ 2020/ кроссовер 5 дв. (7 мест)/ 2.2 CRDi/ Дизель/ Полный/ Робот/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2019/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2019/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2019/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2019/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2019/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2019/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2019/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2019/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2018/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2018/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2018/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2018/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2018/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2018/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2018/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2018/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2017/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2017/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2017/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2017/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2017/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2017/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2017/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2016/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2016/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2016/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2016/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2016/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2015/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2015/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2015/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2015/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2015/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2015/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2015/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2015/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2014/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2014/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2014/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2014/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2013/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Механика/ , Sorento XM FL/ 2013/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2013/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2013/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2013/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Механика/ , Sorento XM FL/ 2013/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM FL/ 2020/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Sorento Prime UM FL/ 2020/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2020/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2020/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM PE/ 2020/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM PE/ 2020/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM PE/ 2020/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM PE/ 2020/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Sorento Prime UM PE/ 2020/ кроссовер 5 дв. (7 мест)/ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM PE/ 2020/ кроссовер 5 дв. (7 мест)/ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM PE/ 2020/ кроссовер 5 дв. (7 мест)/ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2017/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM FL/ 2017/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM/ 2017/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2017/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM/ 2016/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2016/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM/ 2015/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2015/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM/ 2015/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2014/ кроссовер 5 дв. (7 мест)/ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Soul SK3/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2021/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul SK3/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2021/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul SK3/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2019/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2019/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul SK3/ 2019/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2019/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2019/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul PS FL/ 2019/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2019/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2019/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2019/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul SK3/ 2019/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2017/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2017/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2016/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2016/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2016/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul PS/ 2016/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul PS/ 2016/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS/ 2016/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2016/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2015/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS/ 2015/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2015/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2015/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS/ 2015/ кроссовер 5 дв./ 1.6 CRDi/ Дизель/ Передний/ Автомат/ , Soul PS/ 2015/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2014/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS/ 2014/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2014/ кроссовер 5 дв./ 1.6 CRDi/ Дизель/ Передний/ Автомат/ , Soul PS/ 2013/ кроссовер 5 дв./ 1.6 CRDi/ Дизель/ Передний/ Автомат/ , Soul PS/ 2013/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2022/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2021/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2018/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2018/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage SL FL/ 2016/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2015/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage SL FL/ 2014/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage SL FL/ 2014/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage SL FL/ 2014/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage SL FL/ 2014/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage SL FL/ 2014/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/
## Наименование: S-OIL 7 GOLD №9 PAO C3 5W-30
S-OIL 7 GOLD №9 PAO C3 5W-30
Предназначено для использования в высокофорсированных бензиновых и дизельных двигателях, оборудованных сажевым фильтром и(или) трёхкомпонентным нейтрализатором. Благодаря синтетическому базовому маслу на основе полиальфаолефинов (ПАО) устойчиво к температурным перепадам и сохраняет свои свойства в течение длительного срока. Превосходные очищающие свойства препятствуют образованию нагара и отложений в двигателе.
Подходит моделям с комплектациями: Carnival KA4/ 2022/ кроссвэн (7 мест)/ 2.2 CRDi/ Дизель/ Передний/ Автомат/ , Carnival KA4/ 2022/ кроссвэн (8 мест)/ 2.2 CRDi/ Дизель/ Передний/ Автомат/ , Carnival KA4/ 2022/ кроссвэн (7 мест)/ 2.2 CRDi/ Дизель/ Передний/ Автомат/ , Carnival KA4/ 2022/ кроссвэн (8 мест)/ 2.2 CRDi/ Дизель/ Передний/ Автомат/ , Carnival KA4/ 2021/ кроссвэн (7 мест)/ 2.2 CRDi/ Дизель/ Передний/ Автомат/ , Carnival KA4/ 2021/ кроссвэн (8 мест)/ 2.2 CRDi/ Дизель/ Передний/ Автомат/ , Carnival KA4/ 2021/ кроссвэн (7 мест)/ 2.2 CRDi/ Дизель/ Передний/ Автомат/ , Carnival KA4/ 2021/ кроссвэн (8 мест)/ 2.2 CRDi/ Дизель/ Передний/ Автомат/ , Mohave HM PE2/ 2022/ кроссовер 5 дв. (6 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE2/ 2022/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE2/ 2022/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE2/ 2021/ кроссовер 5 дв. (6 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE2/ 2021/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE2/ 2021/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE2/ 2021/ кроссовер 5 дв. (6 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE2/ 2021/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE2/ 2020/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM FL/ 2020/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM PE/ 2019/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM FL/ 2019/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM FL/ 2018/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM FL/ 2018/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM FL/ 2017/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Mohave HM FL/ 2017/ кроссовер 5 дв. (7 мест)/ 3.0 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2021/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2021/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2019/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2019/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2018/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2018/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2017/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2015/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2015/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2014/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento XM FL/ 2013/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Механика/ , Sorento XM FL/ 2013/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM FL/ 2020/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM PE/ 2020/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM PE/ 2020/ кроссовер 5 дв. (7 мест)/ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM FL/ 2017/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2017/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2016/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2015/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2015/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2014/ кроссовер 5 дв. (7 мест)/ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Soul PS/ 2015/ кроссовер 5 дв./ 1.6 CRDi/ Дизель/ Передний/ Автомат/ , Soul PS/ 2014/ кроссовер 5 дв./ 1.6 CRDi/ Дизель/ Передний/ Автомат/ , Soul PS/ 2013/ кроссовер 5 дв./ 1.6 CRDi/ Дизель/ Передний/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe FL/ 2018/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage SL FL/ 2016/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2015/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage SL FL/ 2014/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/
## Наименование: REPSOL LEADER NEO SP/GF6-6A 0W20
REPSOL LEADER NEO SP/GF6-6A 0W20
Предназначено для использования во всех бензиновых двигателях, включая двигатели с турбонаддувом и системой непосредственного впрыска топлива. Эксклюзивная рецептура 3Е Technology обеспечивает наилучшую защиту двигателя и высочайшую производительность (Технология 3E: Excellent Engine Efficiency = Превосходная Эффективность Двигателя). Благодаря низкой вязкости снижает выбросы CO2 и способствует экономии топлива при нормальных условиях эксплуатации. Обеспечивает уверенный пуск двигателя при отрицательных температурах.
Подходит моделям с комплектациями: Carnival KA4/ 2022/ кроссвэн (7 мест)/ 3.5 MPI/ Бензин/ Передний/ Автомат/ , Carnival KA4/ 2022/ кроссвэн (8 мест)/ 3.5 MPI/ Бензин/ Передний/ Автомат/ , Carnival KA4/ 2021/ кроссвэн (7 мест)/ 3.5 MPI/ Бензин/ Передний/ Автомат/ , Carnival KA4/ 2021/ кроссвэн (8 мест)/ 3.5 MPI/ Бензин/ Передний/ Автомат/ , Carnival KA4/ 2021/ кроссвэн (7 мест)/ 3.5 MPI/ Бензин/ Передний/ Автомат/ , Carnival KA4/ 2021/ кроссвэн (8 мест)/ 3.5 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD PE/ 2022/ хэтчбэк 5 дв./ 1.5 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD PE/ 2021/ хэтчбэк 5 дв./ 1.5 T-GDI/ Бензин/ Передний/ Робот/ , Ceed SW CD PE/ 2022/ универсал 5 дв./ 1.5 T-GDI/ Бензин/ Передний/ Робот/ , Ceed SW CD PE/ 2022/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed SW CD PE/ 2021/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed SW CD PE/ 2021/ универсал 5 дв./ 1.5 T-GDI/ Бензин/ Передний/ Робот/ , Cerato BD/ 2022/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato BD/ 2022/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato BD/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato BD/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2022/ седан 4 дв./ 2.5 GDI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2022/ седан 4 дв./ 2.5 GDI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2021/ седан 4 дв./ 2.5 GDI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2021/ седан 4 дв./ 2.5 GDI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2020/ седан 4 дв./ 2.5 GDI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2020/ седан 4 дв./ 2.5 GDI/ Бензин/ Передний/ Автомат/ , Picanto JA PE/ 2022/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA PE/ 2022/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA PE/ 2022/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA PE/ 2022/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA PE/ 2021/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA PE/ 2021/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA PE/ 2021/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA PE/ 2021/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA PE/ 2020/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA PE/ 2019/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA/ 2019/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA/ 2018/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA/ 2018/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA/ 2017/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA/ 2017/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Sorento MQ4/ 2022/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2022/ кроссовер 5 дв./ 2.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2022/ кроссовер 5 дв./ 2.5 MPI/ Бензин/ Передний/ Автомат/ , Sorento MQ4/ 2022/ кроссовер 5 дв. (7 мест)/ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2022/ кроссовер 5 дв. (7 мест)/ 2.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2022/ кроссовер 5 дв. (7 мест)/ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв./ 2.5 MPI/ Бензин/ Передний/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв./ 2.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв. (7 мест)/ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв. (7 мест)/ 2.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв./ 2.5 MPI/ Бензин/ Передний/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв./ 2.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв. (7 мест)/ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв. (7 мест)/ 2.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2020/ кроссовер 5 дв./ 2.5 MPI/ Бензин/ Передний/ Автомат/ , Sorento MQ4/ 2020/ кроссовер 5 дв./ 2.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2020/ кроссовер 5 дв. (7 мест)/ 2.5 MPI/ Бензин/ Полный/ Автомат/ , Sportage NQ5/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage NQ5/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage NQ5/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage NQ5/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage NQ5/ 2022/ кроссовер 5 дв./ 2.5 GDI/ Бензин/ Полный/ Автомат/ , XCeed CD CUV/ 2022/ кроссовер/ 1.5 T-GDI/ Бензин/ Передний/ Робот/ , XCeed CD CUV/ 2021/ кроссовер/ 1.5 T-GDI/ Бензин/ Передний/ Робот/
## Наименование: S-OIL 7 RED №9 SP 0W-20
S-OIL 7 RED №9 SP 0W-20
Предназначено для использования во всех бензиновых двигателях, включая двигатели с турбонаддувом и системой непосредственного впрыска топлива. Предотвращает возникновение проблем, связанных с ранним зажиганием на низких оборотах в двигателях T-GDI. Обеспечивает превосходные эксплуатационные характеристики, благодаря стабильным свойствам базового масла и высокой степени его очистки и уверенный пуск двигателя при отрицательных температурах. Новейший пакет присадок препятствует образованию нагара и отложений в двигателе.
Подходит моделям с комплектациями: Carnival KA4/ 2022/ кроссвэн (7 мест)/ 3.5 MPI/ Бензин/ Передний/ Автомат/ , Carnival KA4/ 2022/ кроссвэн (8 мест)/ 3.5 MPI/ Бензин/ Передний/ Автомат/ , Carnival KA4/ 2021/ кроссвэн (7 мест)/ 3.5 MPI/ Бензин/ Передний/ Автомат/ , Carnival KA4/ 2021/ кроссвэн (8 мест)/ 3.5 MPI/ Бензин/ Передний/ Автомат/ , Carnival KA4/ 2021/ кроссвэн (7 мест)/ 3.5 MPI/ Бензин/ Передний/ Автомат/ , Carnival KA4/ 2021/ кроссвэн (8 мест)/ 3.5 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD PE/ 2022/ хэтчбэк 5 дв./ 1.5 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD PE/ 2021/ хэтчбэк 5 дв./ 1.5 T-GDI/ Бензин/ Передний/ Робот/ , Ceed SW CD PE/ 2022/ универсал 5 дв./ 1.5 T-GDI/ Бензин/ Передний/ Робот/ , Ceed SW CD PE/ 2021/ универсал 5 дв./ 1.5 T-GDI/ Бензин/ Передний/ Робот/ , K5 DL3/ 2022/ седан 4 дв./ 2.5 GDI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2022/ седан 4 дв./ 2.5 GDI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2021/ седан 4 дв./ 2.5 GDI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2021/ седан 4 дв./ 2.5 GDI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2020/ седан 4 дв./ 2.5 GDI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2020/ седан 4 дв./ 2.5 GDI/ Бензин/ Передний/ Автомат/ , Picanto JA PE/ 2022/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA PE/ 2022/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA PE/ 2022/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA PE/ 2022/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA PE/ 2021/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA PE/ 2021/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA PE/ 2021/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA PE/ 2021/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA PE/ 2020/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA PE/ 2019/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA/ 2019/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA/ 2018/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA/ 2018/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA/ 2017/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Picanto JA/ 2017/ хэтчбэк 5 дв./ 1.0 MPI/ Бензин/ Передний/ Механика/ , Sorento MQ4/ 2022/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2022/ кроссовер 5 дв./ 2.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2022/ кроссовер 5 дв./ 2.5 MPI/ Бензин/ Передний/ Автомат/ , Sorento MQ4/ 2022/ кроссовер 5 дв. (7 мест)/ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2022/ кроссовер 5 дв. (7 мест)/ 2.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2022/ кроссовер 5 дв. (7 мест)/ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв./ 2.5 MPI/ Бензин/ Передний/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв./ 2.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв. (7 мест)/ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв. (7 мест)/ 2.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв./ 2.5 MPI/ Бензин/ Передний/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв./ 2.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв. (7 мест)/ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2021/ кроссовер 5 дв. (7 мест)/ 2.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2020/ кроссовер 5 дв./ 2.5 MPI/ Бензин/ Передний/ Автомат/ , Sorento MQ4/ 2020/ кроссовер 5 дв./ 2.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento MQ4/ 2020/ кроссовер 5 дв. (7 мест)/ 2.5 MPI/ Бензин/ Полный/ Автомат/ , Sportage NQ5/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage NQ5/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage NQ5/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage NQ5/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage NQ5/ 2022/ кроссовер 5 дв./ 2.5 GDI/ Бензин/ Полный/ Автомат/ , XCeed CD CUV/ 2022/ кроссовер/ 1.5 T-GDI/ Бензин/ Передний/ Робот/ , XCeed CD CUV/ 2021/ кроссовер/ 1.5 T-GDI/ Бензин/ Передний/ Робот/
## Наименование: S-OIL 7 RED №9 SP 5W-20
S-OIL 7 RED №9 SP 5W-20
Предназначено для использования во всех бензиновых двигателях, включая двигатели с турбонаддувом и системой непосредственного впрыска топлива. Предотвращает возникновение проблем, связанных с ранним зажиганием на низких оборотах в двигателях T-GDI. Обеспечивает превосходные эксплуатационные характеристики, благодаря стабильным свойствам базового масла и высокой степени его очистки и улучшенную топливную эффективность. Новейший пакет присадок препятствует образованию нагара и отложений в двигателе.
Подходит моделям с комплектациями: Ceed CD PE/ 2022/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD PE/ 2021/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD/ 2021/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD/ 2021/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Ceed CD/ 2019/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Ceed CD/ 2019/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD/ 2019/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Ceed CD/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Ceed CD/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed CD/ 2018/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2018/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2016/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2015/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2018/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2016/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2015/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Ceed SW CD PE/ 2022/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed SW CD/ 2022/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed SW CD PE/ 2021/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed SW CD/ 2021/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Ceed SW CD/ 2021/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed SW CD/ 2021/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato BD/ 2022/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato BD/ 2022/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato BD/ 2022/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato BD/ 2022/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato BD/ 2022/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato BD/ 2022/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato BD/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato BD/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato BD/ 2021/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2021/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2021/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2016/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2016/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2015/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2015/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2015/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2014/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2014/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2014/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2014/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2014/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2014/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2014/ купе/ 2.0 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2014/ купе/ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2013/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2013/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2013/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2013/ купе/ 2.0 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2013/ купе/ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2013/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2013/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2013/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2022/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2021/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2021/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , K5 DL3/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF PE/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF PE/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF FL/ 2019/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2019/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF FL/ 2019/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF FL/ 2019/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2018/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF FL/ 2018/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2018/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF/ 2018/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF/ 2017/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF/ 2017/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2016/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Optima JF/ 2016/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD FL/ 2016/ хэтчбэк 3 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB PE/ 2022/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB PE/ 2022/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB PE/ 2021/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB PE/ 2021/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB PE/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB PE/ 2021/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB PE/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB PE/ 2021/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB PE/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB PE/ 2020/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB PE/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB PE/ 2020/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2020/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2020/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2019/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2019/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2019/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2019/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2018/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2018/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2018/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2018/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2017/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2017/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X FB PE/ 2022/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X FB PE/ 2022/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X FB PE/ 2021/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X FB PE/ 2021/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X FB PE/ 2021/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X FB PE/ 2021/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X FB PE/ 2021/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X FB PE/ 2021/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X FB PE/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X FB PE/ 2020/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X FB PE/ 2020/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X FB PE/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2020/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2020/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2017/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2017/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Вариатор/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Вариатор/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Автомат/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Механика/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Вариатор/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Вариатор/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Механика/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Автомат/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Механика/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Автомат/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Механика/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Вариатор/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Вариатор/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Автомат/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Механика/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Вариатор/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Вариатор/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Автомат/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Автомат/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Механика/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Вариатор/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Вариатор/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Механика/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Автомат/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Вариатор/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Вариатор/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Вариатор/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Вариатор/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Автомат/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Механика/ , Seltos SP2/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Вариатор/ , Sorento XM FL/ 2021/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2021/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2021/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2021/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2020/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2019/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2019/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2019/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2019/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2019/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2019/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2018/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2018/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2018/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2018/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2018/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2018/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2017/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2017/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2017/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2017/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2017/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2017/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2016/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2016/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2016/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2016/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2016/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2015/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2015/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2015/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2015/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2015/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2015/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2014/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2014/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2014/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento XM FL/ 2013/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Механика/ , Sorento XM FL/ 2013/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Передний/ Автомат/ , Sorento XM FL/ 2013/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Механика/ , Sorento XM FL/ 2013/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2020/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM PE/ 2020/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM PE/ 2020/ кроссовер 5 дв. (7 мест)/ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2017/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM/ 2016/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM/ 2015/ кроссовер 5 дв./ 2.4 MPI/ Бензин/ Полный/ Автомат/ , Soul SK3/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2019/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2019/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2019/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2019/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2019/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2019/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2017/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2016/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2016/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS/ 2016/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2015/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS/ 2015/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2015/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS/ 2015/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2014/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS/ 2014/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2013/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage SL FL/ 2014/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage SL FL/ 2014/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage SL FL/ 2014/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage SL FL/ 2014/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/
## Наименование: REPSOL ELITE EVOLUTION V C2 0W30
REPSOL ELITE EVOLUTION V C2 0W30
Предназначено для использования в высокофорсированных бензиновых и дизельных двигателях, оборудованных сажевым фильтром. Минимизирует образование шлама и отложений и максимально защищает двигатель от износа. Снижает эффект повышения расхода топлива и снижения производительности двигателя вызываемых скоплением сажи в фильтре. Обеспечивает уверенный пуск двигателя при отрицательных температурах.
Подходит моделям с комплектациями: Ceed CD/ 2021/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD/ 2019/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD/ 2018/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD/ 2018/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed SW CD/ 2021/ универсал 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2021/ универсал/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2021/ универсал/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2020/ универсал/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2019/ универсал/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Sorento Prime UM/ 2017/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2016/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2015/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2015/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2014/ кроссовер 5 дв. (7 мест)/ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage SL FL/ 2016/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2015/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage SL FL/ 2014/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Stinger CK PE/ 2022/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK PE/ 2022/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK PE/ 2022/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK PE/ 2021/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK PE/ 2021/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK PE/ 2021/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK PE/ 2021/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2020/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2020/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2019/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2019/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2019/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2019/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2018/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2018/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2018/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2018/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2017/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , XCeed CD CUV/ 2021/ кроссовер/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , XCeed CD CUV/ 2021/ кроссовер/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , XCeed CD CUV/ 2020/ кроссовер/ 1.4 T-GDI/ Бензин/ Передний/ Робот/
## Наименование: S-OIL 7 GOLD №9 A5/B5 5W-30
S-OIL 7 GOLD №9 A5/B5 5W-30
Предназначено для удовлетворения самых строгих требований к бензиновым и дизельным двигателям, которые требуют соответствия стандартам ACEA A5/B5. Способствует экономии топлива и снижению выбросов CO2 за счёт улучшенных фрикционных свойств. Благодаря синтетическому базовому маслу высокой степени очистки, сохраняет свои свойства в течение длительного срока. Подходит для автомобилей с турбонаддувом и системой непосредственного впрыска топлива.
Подходит моделям с комплектациями: Ceed CD/ 2021/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD/ 2019/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD/ 2019/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Ceed CD/ 2018/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD/ 2018/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD/ 2018/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2018/ хэтчбэк 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd JD FL/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD FL/ 2018/ хэтчбэк 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD FL/ 2017/ хэтчбэк 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd JD FL/ 2016/ хэтчбэк 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd JD FL/ 2016/ хэтчбэк 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2016/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2016/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD FL/ 2015/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2015/ хэтчбэк 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , Cee'd JD FL/ 2015/ хэтчбэк 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd JD FL/ 2015/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD/ 2015/ хэтчбэк 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd JD/ 2015/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD/ 2015/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2015/ хэтчбэк 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2015/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2014/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD/ 2014/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2014/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2014/ хэтчбэк 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2014/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2014/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2014/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD/ 2014/ хэтчбэк 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2013/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2013/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2013/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD/ 2013/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD/ 2013/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2013/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2012/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2012/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD/ 2012/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2012/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd JD/ 2012/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd JD/ 2012/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2019/ универсал 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd SW JD FL/ 2018/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD FL/ 2018/ универсал 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd SW JD FL/ 2018/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2017/ универсал 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd SW JD FL/ 2016/ универсал 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd SW JD FL/ 2016/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD FL/ 2016/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2015/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD FL/ 2015/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD FL/ 2015/ универсал 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd SW JD/ 2015/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD/ 2015/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2015/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2015/ универсал 5 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , Cee'd SW JD/ 2014/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2014/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD/ 2014/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2014/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2014/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD/ 2014/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2013/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2013/ универсал 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2013/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD/ 2013/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2013/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD/ 2012/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cee'd SW JD/ 2012/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cee'd SW JD/ 2012/ универсал 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Ceed SW CD/ 2021/ универсал 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Cerato Classic YD FL/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2021/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2021/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2020/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2019/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2018/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2017/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD FL/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD FL/ 2016/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2016/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2015/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2015/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2015/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2014/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2014/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2014/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2014/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2014/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2014/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2014/ купе/ 2.0 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2014/ купе/ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2013/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2013/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2013/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2013/ купе/ 2.0 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2013/ купе/ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2013/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Cerato Classic YD/ 2013/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Cerato Classic YD/ 2013/ седан 4 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , K9 RJ PE/ 2022/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2022/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2021/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2021/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2020/ седан 4 дв./ 5.0 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2020/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2020/ седан 4 дв./ 5.0 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2020/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2019/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2019/ седан 4 дв./ 5.0 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2019/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2018/ седан 4 дв./ 5.0 GDI/ Бензин/ Полный/ Автомат/ , K900 RJ/ 2018/ седан 4 дв./ 3.3 GDI/ Бензин/ Полный/ Автомат/ , Optima JF FL/ 2020/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2020/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF PE/ 2020/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF PE/ 2020/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2019/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2019/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2019/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2019/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2018/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF FL/ 2018/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2018/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2018/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2017/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2016/ седан 4 дв./ 2.0 T-GDI/ Бензин/ Передний/ Автомат/ , Optima JF/ 2016/ седан 4 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Picanto JA PE/ 2020/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Механика/ , Picanto JA PE/ 2020/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA PE/ 2019/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA/ 2019/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA/ 2019/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA/ 2018/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA/ 2018/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA/ 2017/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , Picanto JA/ 2017/ хэтчбэк 5 дв./ 1.2 MPI/ Бензин/ Передний/ Автомат/ , ProCeed CD/ 2021/ универсал/ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2021/ универсал/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2021/ универсал/ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2021/ универсал/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2020/ универсал/ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2020/ универсал/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2020/ универсал/ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2019/ универсал/ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2019/ универсал/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2018/ универсал/ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , ProCee'd JD FL/ 2018/ хэтчбэк 3 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , ProCee'd JD FL/ 2018/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD FL/ 2018/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2018/ хэтчбэк 3 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , ProCee'd JD FL/ 2017/ хэтчбэк 3 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2017/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2017/ хэтчбэк 3 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , ProCee'd JD FL/ 2017/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD FL/ 2017/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2017/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD FL/ 2017/ хэтчбэк 3 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , ProCee'd JD FL/ 2017/ хэтчбэк 3 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2016/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2016/ хэтчбэк 3 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , ProCee'd JD FL/ 2016/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD FL/ 2016/ хэтчбэк 3 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2015/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2015/ хэтчбэк 3 дв./ 1.6 GDI/ Бензин/ Передний/ Робот/ , ProCee'd JD FL/ 2015/ хэтчбэк 3 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , ProCee'd JD FL/ 2015/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD/ 2015/ хэтчбэк 3 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD/ 2015/ хэтчбэк 3 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , ProCee'd JD/ 2015/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD/ 2015/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD/ 2014/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD/ 2014/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD/ 2014/ хэтчбэк 3 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , ProCee'd JD/ 2014/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD/ 2014/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD/ 2014/ хэтчбэк 3 дв./ 1.6 T-GDI/ Бензин/ Передний/ Механика/ , ProCee'd JD/ 2013/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , ProCee'd JD/ 2013/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , ProCee'd JD/ 2013/ хэтчбэк 3 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB PE/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB PE/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB PE/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB PE/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2020/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2019/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2018/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X FB PE/ 2021/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X FB PE/ 2021/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X FB PE/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X FB PE/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2019/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2018/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X-Line FB/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Автомат/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Механика/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Вариатор/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Вариатор/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Механика/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Автомат/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Механика/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Автомат/ , Seltos SP2/ 2022/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Механика/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Автомат/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Механика/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Вариатор/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Вариатор/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Автомат/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Автомат/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Механика/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Вариатор/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Seltos SP2/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Вариатор/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Механика/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Автомат/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Вариатор/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Вариатор/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Вариатор/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Вариатор/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Автомат/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Полный/ Механика/ , Seltos SP2/ 2020/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Seltos SP2/ 2019/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Seltos SP2/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Вариатор/ , Sorento Prime UM FL/ 2020/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Sorento Prime UM FL/ 2020/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2020/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM PE/ 2020/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM PE/ 2020/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM PE/ 2020/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Sorento Prime UM PE/ 2020/ кроссовер 5 дв. (7 мест)/ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM PE/ 2020/ кроссовер 5 дв. (7 мест)/ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2019/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Передний/ Автомат/ , Sorento Prime UM FL/ 2018/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM FL/ 2017/ кроссовер 5 дв./ 3.5 MPI/ Бензин/ Полный/ Автомат/ , Sorento Prime UM/ 2017/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Soul SK3/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul SK3/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2019/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2019/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul PS FL/ 2019/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2019/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul SK3/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul PS FL/ 2018/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS FL/ 2017/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2017/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2017/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2016/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2016/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS FL/ 2016/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul PS/ 2016/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul PS/ 2016/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS/ 2016/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2016/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2015/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS/ 2015/ кроссовер 5 дв./ 1.6 GDI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2015/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2015/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS/ 2015/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2014/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Soul PS/ 2014/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Soul PS/ 2013/ кроссовер 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2022/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2022/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2021/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2021/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2020/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2019/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe FL/ 2018/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sportage QLe FL/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.4 GDI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage QLe/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage SL FL/ 2014/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Механика/ , Sportage SL FL/ 2014/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Механика/ , Sportage SL FL/ 2014/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Полный/ Автомат/ , Sportage SL FL/ 2014/ кроссовер 5 дв./ 2.0 MPI/ Бензин/ Передний/ Автомат/ , Stinger CK PE/ 2022/ лифтбек/ 3.3 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK PE/ 2022/ лифтбек/ 3.3 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK PE/ 2022/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK PE/ 2021/ лифтбек/ 3.3 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK PE/ 2021/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK PE/ 2021/ лифтбек/ 3.3 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK PE/ 2021/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2020/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2020/ лифтбек/ 3.3 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2020/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2019/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2019/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2019/ лифтбек/ 3.3 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2019/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2019/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2019/ лифтбек/ 3.3 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2018/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2018/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2018/ лифтбек/ 3.3 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2018/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2018/ лифтбек/ 3.3 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2018/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2017/ лифтбек/ 3.3 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2017/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , XCeed CD CUV/ 2022/ кроссовер/ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , XCeed CD CUV/ 2021/ кроссовер/ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , XCeed CD CUV/ 2021/ кроссовер/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , XCeed CD CUV/ 2021/ кроссовер/ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , XCeed CD CUV/ 2021/ кроссовер/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , XCeed CD CUV/ 2020/ кроссовер/ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , XCeed CD CUV/ 2020/ кроссовер/ 1.4 T-GDI/ Бензин/ Передний/ Робот/
## Наименование: S-OIL 7 GOLD №9 C2 0W-30
S-OIL 7 GOLD №9 C2 0W-30
Предназначено для использования в высокофорсированных бензиновых и дизельных двигателях, оборудованных сажевым фильтром и(или) трёхкомпонентным нейтрализатором. Благодаря синтетическому базовому маслу высокой степени очистки, сохраняет свои свойства в течение длительного срока. Расширенный диапазон вязкости обеспечивает уверенный пуск двигателя при отрицательных температурах.
Подходит моделям с комплектациями: Ceed CD/ 2021/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD/ 2019/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD/ 2018/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed CD/ 2018/ хэтчбэк 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Ceed SW CD/ 2021/ универсал 5 дв./ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2021/ универсал/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2021/ универсал/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2020/ универсал/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , ProCeed CD/ 2019/ универсал/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , Sorento Prime UM/ 2017/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2016/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2015/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2015/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sorento Prime UM/ 2014/ кроссовер 5 дв. (7 мест)/ 2.2 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage SL FL/ 2016/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage QLe/ 2015/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage SL FL/ 2015/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Sportage SL FL/ 2014/ кроссовер 5 дв./ 2.0 CRDi/ Дизель/ Полный/ Автомат/ , Stinger CK PE/ 2022/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK PE/ 2022/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK PE/ 2022/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK PE/ 2021/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK PE/ 2021/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK PE/ 2021/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK PE/ 2021/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2020/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2020/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2019/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2019/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2019/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2019/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2018/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2018/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2018/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , Stinger CK/ 2018/ лифтбек/ 2.0 T-GDI/ Бензин/ Задний/ Автомат/ , Stinger CK/ 2017/ лифтбек/ 2.0 T-GDI/ Бензин/ Полный/ Автомат/ , XCeed CD CUV/ 2021/ кроссовер/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , XCeed CD CUV/ 2021/ кроссовер/ 1.4 T-GDI/ Бензин/ Передний/ Робот/ , XCeed CD CUV/ 2020/ кроссовер/ 1.4 T-GDI/ Бензин/ Передний/ Робот/
## Наименование: S-OIL 7 RED №9 SP 5W-30
S-OIL 7 RED №9 SP 5W-30
Предназначено для использования во всех бензиновых двигателях, включая двигатели с турбонаддувом и системой непосредственного впрыска топлива. Предотвращает возникновение проблем, связанных с ранним зажиганием на низких оборотах в двигателях T-GDI. Обеспечивает превосходные эксплуатационные характеристики, благодаря стабильным свойствам базового масла и высокой степени его очистки. Новейший пакет присадок препятствует образованию нагара и отложений в двигателе.
Подходит моделям с комплектациями: Rio FB PE/ 2022/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB PE/ 2022/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio FB PE/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio FB PE/ 2021/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2017/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2017/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2017/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2017/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2017/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2017/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2016/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2016/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2016/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2016/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2016/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2016/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2016/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2016/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2016/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2016/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2016/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2016/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2016/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2015/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2015/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2015/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2015/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2015/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2015/ седан 4 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2015/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2015/ седан 4 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2015/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2015/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2015/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2015/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2014/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2014/ хэтчбэк 5 дв./ 1.4 MPI/ Бензин/ Передний/ Автомат/ , Rio QB FL/ 2014/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio QB FL/ 2014/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X FB PE/ 2022/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X FB PE/ 2022/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X FB PE/ 2021/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/ , Rio X FB PE/ 2021/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/ , Rio X-Line FB PE/ 2021/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Механика/
## Наименование: S-OIL 7 GOLD №9 C3 5W-30
S-OIL 7 GOLD №9 C3 5W-30
Предназначено для использования в высокофорсированных бензиновых и дизельных двигателях, оборудованных сажевым фильтром или трехкомпонентным нейтрализатором. Благодаря синтетическому базовому маслу высокой степени очистки, сохраняет свои свойства в течение длительного срока. Превосходные очищающие свойства препятствуют образованию нагара и отложений в двигателе.
Подходит моделям с комплектациями: Sorento MQ4/ 2022/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Робот/ , Sorento MQ4/ 2022/ кроссовер 5 дв. (7 мест)/ 2.2 CRDi/ Дизель/ Полный/ Робот/ , Sorento MQ4/ 2021/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Робот/ , Sorento MQ4/ 2021/ кроссовер 5 дв. (7 мест)/ 2.2 CRDi/ Дизель/ Полный/ Робот/ , Sorento MQ4/ 2021/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Робот/ , Sorento MQ4/ 2021/ кроссовер 5 дв. (7 мест)/ 2.2 CRDi/ Дизель/ Полный/ Робот/ , Sorento MQ4/ 2020/ кроссовер 5 дв./ 2.2 CRDi/ Дизель/ Полный/ Робот/ , Sorento MQ4/ 2020/ кроссовер 5 дв. (7 мест)/ 2.2 CRDi/ Дизель/ Полный/ Робот/
## Наименование: S-OIL 7 RED №9 SP 5W-40
S-OIL 7 RED №9 SP 5W-40
Предназначено для использования во всех бензиновых двигателях, включая двигатели с турбонаддувом и системой непосредственного впрыска топлива. Предотвращает возникновение проблем, связанных с ранним зажиганием на низких оборотах в двигателях T-GDI. Обеспечивает превосходные эксплуатационные характеристики, благодаря стабильным свойствам базового масла и высокой степени его очистки и повышенную прочность масляной пленки в рабочем состоянии. Новейший пакет присадок препятствует образованию нагара и отложений в двигателе.
Подходит моделям с комплектациями: Soul SK3/ 2021/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul SK3/ 2021/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul SK3/ 2020/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul SK3/ 2019/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Soul SK3/ 2019/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Передний/ Робот/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Sportage QLe/ 2018/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Sportage QLe/ 2017/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/ , Sportage QLe/ 2016/ кроссовер 5 дв./ 1.6 T-GDI/ Бензин/ Полный/ Робот/
"""

In [ ]:
with open("/content/drive/MyDrive/KIA_PARSER/oils.txt", "w") as f:
    f.write(OILS_TEXT)

In [ ]:
dict={}
with open("/content/drive/MyDrive/KIA_PARSER/oils.txt", "r") as fr:
    key=None
    desc=None
    for line in fr.readlines():
        if line.startswith("## Наименование: "):
            key=line[len("## Наименование: "): len(line)-1]
            print(key)
            desc=""
        else:
            models = line.split(",")
            for m in models:
                values = m.split("/")
                if key is not None and len(values) > 4:
                    """Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/"""
                    val = values[0] + " с типом двигателя " + values[4]
                    if val in dict:
                        if len(dict[val]) < 3: dict[val].append(key + " (" + desc + ")")
                    else:
                        dict[val] = [key + " (" + desc + ")"]
                else:
                    desc = line.replace("\n", "   ")

with open("/content/drive/MyDrive/KIA_PARSER/oils_database_new.txt", "w") as fw:
    for k in dict:
        fw.write("## Какое масло подходит " + k + "? ")
        fw.write("\n")
        fw.write("Какое масло подходит " + k + "? " + ",".join(dict[k]))
        fw.write("\n")

with open("/content/drive/MyDrive/KIA_PARSER/oils_database_new.txt", "r") as f:
    for line in f.readlines():
        print(line)

REPSOL ELITE EVOLUTION LONG LIFE C3/SN 5W30
S-OIL 7 GOLD №9 PAO C3 5W-30
REPSOL LEADER NEO SP/GF6-6A 0W20
S-OIL 7 RED №9 SP 0W-20
S-OIL 7 RED №9 SP 5W-20
REPSOL ELITE EVOLUTION V C2 0W30
S-OIL 7 GOLD №9 A5/B5 5W-30
S-OIL 7 GOLD №9 C2 0W-30
S-OIL 7 RED №9 SP 5W-30
S-OIL 7 GOLD №9 C3 5W-30
S-OIL 7 RED №9 SP 5W-40
## Какое масло подходит Подходит моделям с комплектациями: Carnival KA4 с типом двигателя  Дизель? 

Какое масло подходит Подходит моделям с комплектациями: Carnival KA4 с типом двигателя  Дизель? REPSOL ELITE EVOLUTION LONG LIFE C3/SN 5W30 (Предназначено для использования в высокофорсированных бензиновых и дизельных двигателях, оборудованных сажевым фильтром. Благодаря высокому качеству, данное масло выделяется низким образованием отложений. Снижает трение, защищает двигатель от износа и имеет большую стойкость к окислению, что позволяет сохранять все свойства в течение длительного срока.   ),S-OIL 7 GOLD №9 PAO C3 5W-30 (Предназначено для использования в высокофорсированных бе

In [ ]:
print("OK")

OK


In [ ]:
my_dict={}
with open("/content/drive/MyDrive/KIA_PARSER/oils.txt", "r") as fr:
    my_key=None
    my_desc=None
    my_first_lines = []
    my_lines = []
    first = None
    for line in fr.readlines():
        if line.startswith("## Наименование: "):
            for my_line in ['   '.join(my_lines)]:
                models = my_line.split(",")
                for m in models:
                    my_values = m.split("/")
                    if my_key is not None and len(my_values) > 1:
                        """Ceed CD/ 2020/ хэтчбэк 5 дв./ 1.6 MPI/ Бензин/ Передний/ Автомат/"""
                        my_short_key = ""
                        my_val = 'Таблица: "Масла", Масло: "' + my_key + '"'
                        if my_key is not None and len(my_values) > 0:
                            my_val += "," + 'Модель: "' + my_values[0].replace("\n", "   ") + '"'
                            my_short_key += "," + 'Модель: "' + my_values[0].replace("\n", "   ") + '"'
                        if my_key is not None and len(my_values) > 4:
                            my_val += "," + 'Топливо: "' + my_values[4].replace("\n", "   ") + '"'
                            my_short_key += "," + 'Топливо: "' + my_values[4].replace("\n", "   ") + '"'
                        if my_key is not None and len(my_values) > 1:
                            my_val += "," + 'Пример Года Выпуска: "' + my_values[1].replace("\n", "   ") + '"'
                        if my_key is not None and len(my_values) > 2:
                            my_val += "," + 'Пример Типа кузова: "' + my_values[2].replace("\n", "   ") + '"'
                        if my_key is not None and len(my_values) > 3:
                            my_val += "," + 'Пример Двигателя: "' + my_values[3].replace("\n", "   ") + '"'
                        if my_key is not None and len(my_values) > 5:
                           my_val += "," + 'Пример Привода: "' + my_values[5].replace("\n", "   ") + '"'
                        if my_key is not None and len(my_values) > 6:
                           my_val += "," + 'Пример Коробки: "' + my_values[6].replace("\n", "   ") + '"'
                        my_dict[my_key + " - " + my_short_key] = my_val + ',' + 'Описание: "' + my_desc + '"'
                    else:
                        my_desc = '   '.join(my_first_lines).replace("/", "   ").replace("\n", "   ")
            my_lines = []

            my_key=line[len("## Наименование: "): len(line)-1]
            print(my_key)
            my_desc=""
            first = True
        else:
            index = line.find("Подходит моделям с ")
            if index >= 0: first = False
            if first:
                  my_first_lines.append(line[:index] if index >= 0 else line)
            my_lines.append(line[index:] if index >=0 else line)


with open("/content/drive/MyDrive/KIA_PARSER/oils_database.txt", "w") as fw:
    for k in my_dict:
        fw.write("## Таблица Масла. Строка " + k)
        fw.write("\n")
        fw.write(my_dict[k])
        fw.write("\n")

with open("/content/drive/MyDrive/KIA_PARSER/oils_database.txt", "r") as f:
    for line in f.readlines():
        print(line)

REPSOL ELITE EVOLUTION LONG LIFE C3/SN 5W30
S-OIL 7 GOLD №9 PAO C3 5W-30
REPSOL LEADER NEO SP/GF6-6A 0W20
S-OIL 7 RED №9 SP 0W-20
S-OIL 7 RED №9 SP 5W-20
REPSOL ELITE EVOLUTION V C2 0W30
S-OIL 7 GOLD №9 A5/B5 5W-30
S-OIL 7 GOLD №9 C2 0W-30
S-OIL 7 RED №9 SP 5W-30
S-OIL 7 GOLD №9 C3 5W-30
S-OIL 7 RED №9 SP 5W-40
## Таблица Масла. Строка REPSOL ELITE EVOLUTION LONG LIFE C3/SN 5W30 - ,Модель: "REPSOL ELITE EVOLUTION LONG LIFE C3"

Таблица: "Масла", Масло: "REPSOL ELITE EVOLUTION LONG LIFE C3/SN 5W30",Модель: "REPSOL ELITE EVOLUTION LONG LIFE C3",Пример Года Выпуска: "SN 5W30      Предназначено для использования в высокофорсированных бензиновых и дизельных двигателях",Описание: ""

## Таблица Масла. Строка REPSOL ELITE EVOLUTION LONG LIFE C3/SN 5W30 - ,Модель: " что позволяет сохранять все свойства в течение длительного срока.      Подходит моделям с комплектациями: Carnival KA4",Топливо: " Дизель"

Таблица: "Масла", Масло: "REPSOL ELITE EVOLUTION LONG LIFE C3/SN 5W30",Модель: " что позвол